## Web Scrapping

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
pd.set_option('display.max_rows', None)
from IPython.display import display, HTML
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from selenium import webdriver
driver = webdriver.Chrome()
from selenium.webdriver.common.by import By

from textblob_de import TextBlobDE

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 100
Current browser version is 114.0.5735.248 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	Ordinal0 [0x00A37413+2389011]
	Ordinal0 [0x009C9F61+1941345]
	Ordinal0 [0x008BC658+837208]
	Ordinal0 [0x008DC773+968563]
	Ordinal0 [0x008D837A+951162]
	Ordinal0 [0x008D5C51+941137]
	Ordinal0 [0x00908C80+1150080]
	Ordinal0 [0x009088DA+1149146]
	Ordinal0 [0x00903F66+1130342]
	Ordinal0 [0x008DE546+976198]
	Ordinal0 [0x008DF456+980054]
	GetHandleVerifier [0x00BE9632+1727522]
	GetHandleVerifier [0x00C9BA4D+2457661]
	GetHandleVerifier [0x00ACEB81+569713]
	GetHandleVerifier [0x00ACDD76+566118]
	Ordinal0 [0x009D0B2B+1968939]
	Ordinal0 [0x009D5988+1989000]
	Ordinal0 [0x009D5A75+1989237]
	Ordinal0 [0x009DECB1+2026673]
	BaseThreadInitThunk [0x757300C9+25]
	RtlGetAppContainerNamedObjectPath [0x77107B1E+286]
	RtlGetAppContainerNamedObjectPath [0x

#### get the page links (need to change the Input)

In [64]:
%%time
# get the pag links
htmls = []

for page in range(1,32): # if the total pages are 3, then input 3+1 = 4  
    
    url = 'https://www.parfumdreams.de/Pflege/Herren?p={}' # the links we wanna scrap.
    htmls.append(url.format(page))
    
    

len(htmls)

CPU times: total: 0 ns
Wall time: 0 ns


31

#### get the product links.

In [65]:
%%time
# get the product links

productlinks = []

for html in htmls:
    

    r = requests.get(html)
    soup = BeautifulSoup(r.content, 'lxml')
    
    for item in soup.find_all('div', class_="product-image c d-12"):
        for link in item.find_all('a', href=True, limit=1):
            productlinks.append('https://www.parfumdreams.de' + link['href'])
    
   
len(productlinks)

CPU times: total: 6.97 s
Wall time: 2min 16s


911

In [66]:

def brand(soup):
    try:
        brand = soup.find('span', itemprop="brand").text
    except AttributeError: 
        return 
    
    return brand

 
def name(soup):
    try:
       
        if soup.find('a', class_="font-headline").text == soup.find('span', itemprop="name").text:
            name = soup.find('span', itemprop="name").text
        else:
            name = soup.find('a', class_="font-headline").text + ' ' +soup.find('span', itemprop="name").text
            
    except AttributeError: 
        return 
    
    return name


def reviews(soup):
    try:
        reviews = soup.find('span', itemprop="reviewCount").text
    except AttributeError:
        return
    
    return reviews


def ratings(soup):
    try:
        ratings = soup.find('span', itemprop="ratingValue").text.replace(",", ".")
    except AttributeError:
        return
    
    return ratings



def price1(row):
    try:
        price1 = row.find('div', class_=re.compile('^font-headline mbs')).find('span', class_="premium-subscription-inactive").text.replace('\r','').replace('\n','').replace('€','').replace('*','').replace(' ','').replace(",", ".")
    except AttributeError:
        return
    
    return price1


def price2(row):
    try:
        price2 = row.find('div', class_=re.compile('^font-headline mbs')).find('span', class_="font-headline-sub no-wrap").text.replace('\r','').replace('\n','').replace('€','').replace('*','').replace(' ','').replace(",", ".").replace("UVP", "")
    except AttributeError:
        return
    
    return price2



def EAN(row):
    try:
        EAN = row.find('meta', itemprop="gtin13")['content']
    except (AttributeError, TypeError):
        return
    
    return EAN


def id(row):
    try:
        id = row.find('div', class_="custom-radio").find('input')
    except AttributeError:
        return
    
    return id['value']


def size(row):
    try:
        size = row.find('span', class_="show-when-no-js").text.replace('ml', '')
    except AttributeError:
        return
    
    return size



In [67]:
%%time

df  = pd.DataFrame(columns=['EAN', 'id', 'brand', 'name','price1','price2','size', "reviews","ratings",'sorting','links'])


for i, link in enumerate(productlinks):

    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'lxml') 
    
    for row in soup.find_all('div',itemprop="offers"):

        product = {

        'EAN': [EAN(row)],
            'id':[id(row)],
        'brand': [brand(soup)],
        'name': [name(soup)],
        'price1': [price1(row)],
        'price2': [price2(row)],
        'size': [size(row)],
        "reviews": [reviews(soup)],
        "ratings": [ratings(soup)],
        'sorting': [i],
        'links' : [link],

        }

        product = pd.DataFrame(product)

        #df.append(product)
        df = df.append(product)

    df.append(df)
    df = df.reset_index().drop('index',  axis=1)
            
df.to_csv('raw.csv', index=False) 
df.shape

CPU times: total: 3min 8s
Wall time: 11min 17s


(1021, 11)

In [49]:
#display(HTML("<div style='height: 500px'>" + df.style.render() + "</div>"))

#### Data Cleaning

In [68]:

# clean th size
for i in range(0,df.shape[0]):
    if '+' in df['size'].iloc[i].split(): 
        df.at[i,'size'] = 'set'
#  clean th size     
for i in range(0,df.shape[0]):
    
    split = df['size'].iloc[i].rsplit('/',1)
    try: 
        df.at[i, 'size'] = split[1]
    except IndexError:
        pass
    try:
        df.at[i,'name'] = df['name'].iloc[i]+' ' +split[0]
    except (IndexError and TypeError):
        pass
    
    df.at[i,'size'] = df['size'].iloc[i].replace('g','')
        


# convert the variables below to float
df[["price1","price2",'ratings','reviews','sorting']] = df[["price1","price2",'ratings','reviews','sorting']].astype(float)

# convert the sizes to float
for i in range(0,df.shape[0]):
    df.at[i,'size'] = pd.to_numeric(df.loc[i]['size'], errors='ignore', downcast='float')
    
# if price2 is empty, then price2 = price1    
for i in range(0, df.shape[0]):
    if pd.isnull(df.loc[i]['price2']):
        df.at[i,'price2'] = df.loc[i]['price1']



#### Parfundreams Customers Comments Scrapping

#### Sentiment Analysis by Textblod

#### the code in the next block only for coloration category

#### add more dimentions

In [69]:


#df['scale polarity (reviews*polarity)'] = df['reviews']*df['polarity']
df['price2 minus price1'] = df['price2'] - df['price1']

df['price1 per 100ml'] = None
df['price2 per 100ml'] = None

for i in range(0,df.shape[0]):
    try:
        df.at[i, 'price1 per 100ml']  = (df['price1'].iloc[i]/df['size'].iloc[i])*100
    except TypeError:    
        df.at[i,'price1 per 100ml'] = None

for i in range(0,df.shape[0]):
    try:
        df.at[i, 'price2 per 100ml']  = (df['price2'].iloc[i]/df['size'].iloc[i])*100
    except TypeError:    
        df.at[i,'price2 per 100ml'] = None


# create a new variable 'brand + name' 
df['brand + name'] = df['brand'] + ' ' + df['name'] 
for i in range(0,df.shape[0]):
    df.at[i,'brand + name'] = str(df['brand + name'].iloc[i]).title().replace('  ',' ')

In [70]:
df['p_category'] = None 
df['pp_category'] = 'Herren'
df['main_category'] = 'Pflege'
df['source'] = 'parfumdreams'
df['discount']  = None
df['deliveryTime'] = 2

In [71]:
df = df[['id', 'EAN', 'brand', 'name', 'brand + name', 'size', 'discount', 'price1', 'price2', 'price2 minus price1', 'price1 per 100ml','price2 per 100ml','reviews', 'ratings', 'sorting', 'p_category', 'pp_category', 'main_category', 'source', 'links', 'deliveryTime']]
df.shape

(1021, 21)

In [72]:
df.to_csv('Herren.csv', index=False)

In [73]:
a = pd.read_csv("Gesicht.csv")
b = pd.read_csv("Sonnen.csv")
c= pd.read_csv("Körper.csv")
d= pd.read_csv("Hand & Fuß.csv")
e= pd.read_csv("Herren.csv")
#f = pd.read_csv("Spezialprodukte.csv")
#g = pd.read_csv("Spray.csv")
#h = pd.read_csv("Wachs.csv")
#i = pd.read_csv("Creme.csv")
#reisegroessen = pd.read_csv("reisegroessen_clean.csv")



df= pd.concat([a,b,c,d,e], ignore_index=True)
df.to_csv('parfumdreams_Pflege.csv', index=False)
df.shape

(13123, 21)

In [ ]:
product = {

    'Name': Bestellnummer(soup),
    'Nennspannung':Nennspannung(soup),
    'Nennleistung': Nennleistung(soup),
    'Farbtemperatur': Farbtemperatur(soup),
    'Nennlichtstrom': Nennlichtstrom(soup),
    'Farbwiedergabeindex_Ra': Farbwiedergabeindex_Ra(soup),
    'Durchmesser': Durchmesser(soup),
    "Länge": Länge(soup),
    "Abstand_Lichtschwerpunkt_LCL": Abstand_Lichtschwerpunkt_LCL(soup),
    'Wendellänge': Wendellänge(soup),
    'Leuchtfeld': Leuchtfeld(soup),
    'Lebensdauer': Lebensdauer(soup),
    'Sockel': Sockel(soup),
    'Mindestdurchmesser_des_Lampengehäuses': Mindestdurchmesser_des_Lampengehäuses(soup),
    'Brennstellung': Brennstellung(soup),
    'Anwendung': Anwendung(soup),
    'Datum_der_Deklaration': Datum_der_Deklarationsoup(soup),
    'Primäre_Erzeugnisnummer': Primäre_Erzeugnisnummer(soup),
    'Stoff_der_Kandidatenliste': Stoff_der_Kandidatenliste(soup),
    'SCIP_Deklarationsnummer': SCIP_Deklarationsnummer(soup),
    'Category': Category(soup),
    'url': url(soup),


}

In [19]:
html = 'https://www.lighting.philips.de//consumer/p/halogen-halogenbrenner/8718291204244'+'/technische-daten'


r = requests.get(html)
soup = BeautifulSoup(r.content, 'lxml')

for i in soup.find_all('dl', class_='p-specs-main-list'):
    for dt, dd in zip(soup.find_all('dt'), soup.find_all('dd')):
        data[dt.text.strip()] = dd.text.strip()



[<dl class="p-specs-main-list">
 <dt>dimmbar
                             
                             
                         </dt>
 <dd>
 <div class=""><span> Ja
                                  </span>
 </div>
 </dd>
 <dt>Lampenform
                             
                             
                         </dt>
 <dd>
 <div class=""><span> Brenner, keine Richtwirkung
                                  </span>
 </div>
 </dd>
 <dt>Sockel
                             
                             
                         </dt>
 <dd>
 <div class=""><span> G9
                                  </span>
 </div>
 </dd>
 <dt>Technologie
                             
                             
                         </dt>
 <dd>
 <div class=""><span> Halogen
                                  </span>
 </div>
 </dd>
 </dl>,
 <dl class="p-specs-main-list">
 <dt>Durchmesser
                             
                             
                         </dt>
 <dd>
 <div clas

In [20]:
r

<Response [200]>

In [31]:

html = 'https://www.lighting.philips.de//consumer/p/halogen-halogenbrenner/8718291204244'+'/technische-daten'

r = requests.get(html)

soup = BeautifulSoup(r.content, 'lxml')
soup.find_all('table')


[]

In [30]:
pd.read_html(str(tables[5]), flavor='bs4')

NameError: name 'tables' is not defined

In [21]:

html = 'https://www.lighting.philips.de//consumer/p/halogen-halogenbrenner/8718291204244'+'/technische-daten'


r = requests.get(html)

soup = BeautifulSoup(html, 'html.parser')
data = []

# find all 'dl' tags
dl_tags = soup.find_all('dl', class_="p-specs-main-list")

for dl in dl_tags:
    temp_dict = {}
    dt_tags = dl.find_all('dt')
    dd_tags = dl.find_all('dd')
    for dt, dd in zip(dt_tags, dd_tags):
        temp_dict[dt.text.strip()] = dd.text.strip()
    data.append(temp_dict)

df = pd.DataFrame(data)

d:\install\conda\lib\site-packages\bs4\__init__.py:404: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


In [22]:
# Send a GET request to the webpage
url = 'https://www.lighting.philips.de//consumer/p/halogen-halogenbrenner/8718291204244'
response = requests.get(url)

# Parse the HTML content of the webpage
soup = BeautifulSoup(response.text, 'html.parser')

# Find all 'dl' tags that contain product details
dl_tags = soup.find_all('dl', class_="p-specs-main-list")

data = []

# Loop over all 'dl' tags
for dl in dl_tags:
    temp_dict = {}
    # Find all 'dt' (term) and 'dd' (description) pairs within the 'dl' tag
    dt_tags = dl.find_all('dt')
    dd_tags = dl.find_all('dd')
    for dt, dd in zip(dt_tags, dd_tags):
        # Remove any leading/trailing whitespace from the text
        term = dt.get_text(strip=True)
        desc = dd.get_text(strip=True)
        temp_dict[term] = desc
    data.append(temp_dict)

# Create a dataframe
df = pd.DataFrame(data)
print(df)

Empty DataFrame
Columns: []
Index: []
